In [55]:
import xgboost as xgb
import pandas as pd
import shutil
import mlflow
import os
import math
import numpy as np
from scipy import stats
from sklearn.model_selection import RandomizedSearchCV

# %run ../utils/utils-graph.ipynb

global baseField
baseField = "like_label"

from datetime import datetime, timedelta
global start_time
start_time = datetime.now()
print(xgb.__version__)
print(start_time)


3.1.2
2025-12-04 15:33:45.664379


In [41]:
# time_train = 'bucketN_0'

# os.system('cp ../data_and_models/'+time_train+'/x_train.pkl  ../data/')
# os.system('cp ../data_and_models/'+time_train+'/x_test.pkl  ../data/')

# os.system('cp ../data_and_models/'+time_train+'/y_train.pkl  ../data/')
# os.system('cp ../data_and_models/'+time_train+'/y_test.pkl  ../data/') 

# os.system('cp ../data_and_models/'+time_train+'/cat_ordinal_encoder.pkl  ../data/') 
# os.system('cp ../data_and_models/'+time_train+'/training_categories_array.pkl  ../data/') 

In [42]:
X_train_path = "./data/x_train.pkl"
y_train_path = "./data/y_train.pkl"

X_train = pd.read_pickle(X_train_path)
# exclude_fields = ["timestamp", "OriginalPaymentId", "void-success", "capture-success", "row_id", "Tenants_tntid"]
#
# X_train.drop(exclude_fields, axis=1, inplace=True)

y_train = pd.read_pickle(y_train_path)[baseField]

In [43]:
print("Train Size X:",len(X_train))
print("Train Size Y:",len(y_train))


Train Size X: 3721
Train Size Y: 3721


In [44]:
y_train.describe()

count    3721.000000
mean        0.332438
std         0.471150
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: like_label, dtype: float64

In [45]:

spw = y_train.value_counts()[0]/y_train.value_counts()[1]
print("======= TRAIN =======")
print("positive: ", y_train.value_counts()[1])
print("negative: ", y_train.value_counts()[0])
spw = math.sqrt(spw)
print("SPW: ",spw)
print("==============")

======= TRAIN =======
positive:  1237
negative:  2484
SPW:  1.417068831910957


In [46]:
features_list = sorted(X_train.columns.tolist())
exclude_list = [baseField, 'mb_id']
features_list = [x for x in features_list if x not in exclude_list]

print(features_list)
print(len(features_list))

['chroma_A', 'chroma_A#', 'chroma_B', 'chroma_C', 'chroma_C#', 'chroma_D', 'chroma_D#', 'chroma_E', 'chroma_F', 'chroma_F#', 'chroma_G', 'chroma_G#', 'genre', 'mfcc_1', 'mfcc_10', 'mfcc_11', 'mfcc_12', 'mfcc_13', 'mfcc_2', 'mfcc_3', 'mfcc_4', 'mfcc_5', 'mfcc_6', 'mfcc_7', 'mfcc_8', 'mfcc_9', 'rms', 'spectral_centroid', 'spectral_rolloff', 'zcr']
30


In [47]:
X_train = X_train[features_list]

In [48]:
X_train.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 3721 entries, 5477 to 5141
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   chroma_A           3721 non-null   float64 
 1   chroma_A#          3721 non-null   float64 
 2   chroma_B           3721 non-null   float64 
 3   chroma_C           3721 non-null   float64 
 4   chroma_C#          3721 non-null   float64 
 5   chroma_D           3721 non-null   float64 
 6   chroma_D#          3721 non-null   float64 
 7   chroma_E           3721 non-null   float64 
 8   chroma_F           3721 non-null   float64 
 9   chroma_F#          3721 non-null   float64 
 10  chroma_G           3721 non-null   float64 
 11  chroma_G#          3721 non-null   float64 
 12  genre              3721 non-null   category
 13  mfcc_1             3721 non-null   float64 
 14  mfcc_10            3721 non-null   float64 
 15  mfcc_11            3721 non-null   float64 
 16  mfcc_12 

In [49]:

### Search Hyper parameters
if False:
    rs_param_grid = {
        'alpha': range(5, 30, 5),
        'colsample_bytree': stats.uniform(0.2, 1),
        'gamma': stats.uniform(0, 0.5),
        'lambda': stats.uniform(0, 1.5),
        'learning_rate' : stats.uniform(0.01, 0.5),
        'max_depth': [5, 6, 7, 8, 9, 10, 11],
        'min_child_weight' : [ 1, 3],
        'n_estimators': range(200, 800, 50),
        'scale_pos_weight': stats.uniform(0.5*spw, 1.5*spw),
        'subsample' : stats.uniform(0.6, 1),
    }
    cv_p = 4
    n_iter_p = 500

    xgb_clf = xgb.XGBClassifier(
    enable_categorical=True,
    verbosity=0,
    )

    xgb_rs = RandomizedSearchCV(
        estimator=xgb_clf,
        param_distributions=rs_param_grid,
        cv=cv_p,
        n_iter=n_iter_p,
        verbose=2
    )
    xgb_rs.fit(X_train, y_train)

    # Print the best parameters and highest accuracy
    print("Best parameters found: ", xgb_rs.best_params_)
    print("Best accuracy found: ", xgb_rs.best_score_)
    best_parameters = xgb_rs.best_params_


Fitting 4 folds for each of 500 candidates, totalling 2000 fits
[CV] END alpha=25, colsample_bytree=0.9924589713136944, gamma=0.28800199180157177, lambda=0.7537213544910234, learning_rate=0.400047118153516, max_depth=7, min_child_weight=3, n_estimators=250, scale_pos_weight=1.307730340526554, subsample=1.4586230697140565; total time=   0.0s
[CV] END alpha=25, colsample_bytree=0.9924589713136944, gamma=0.28800199180157177, lambda=0.7537213544910234, learning_rate=0.400047118153516, max_depth=7, min_child_weight=3, n_estimators=250, scale_pos_weight=1.307730340526554, subsample=1.4586230697140565; total time=   0.0s
[CV] END alpha=25, colsample_bytree=0.9924589713136944, gamma=0.28800199180157177, lambda=0.7537213544910234, learning_rate=0.400047118153516, max_depth=7, min_child_weight=3, n_estimators=250, scale_pos_weight=1.307730340526554, subsample=1.4586230697140565; total time=   0.0s
[CV] END alpha=25, colsample_bytree=0.9924589713136944, gamma=0.28800199180157177, lambda=0.7537213

/home/netmaster/.conda/envs/wuta/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:516: FitFailedWarning: 
1376 fits failed out of a total of 2000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4 fits failed with the following error:
Traceback (most recent call last):
  File "/home/netmaster/.conda/envs/wuta/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/netmaster/.conda/envs/wuta/lib/python3.10/site-packages/xgboost/core.py", line 774, in inner_f
    return func(**kwargs)
  File "/home/netmaster/.conda/envs/wuta/lib/python3.10/site-packages/xgboost/sklearn.py", line 1806, in fit
    self._Booster = train

Best parameters found:  {'alpha': 5, 'colsample_bytree': np.float64(0.6196676211639822), 'gamma': np.float64(0.4594074072727686), 'lambda': np.float64(1.2843342554503443), 'learning_rate': np.float64(0.012866875807393754), 'max_depth': 10, 'min_child_weight': 3, 'n_estimators': 300, 'scale_pos_weight': np.float64(1.30985891486832), 'subsample': np.float64(0.6957922037446137)}
Best accuracy found:  0.7094868507674718


In [56]:
best_parameters = {'alpha': 5, 'colsample_bytree': np.float64(0.6196676211639822), 'gamma': np.float64(0.4594074072727686), 'lambda': np.float64(1.2843342554503443), 'learning_rate': np.float64(0.012866875807393754), 'max_depth': 10, 'min_child_weight': 3, 'n_estimators': 300, 'scale_pos_weight': np.float64(1.30985891486832), 'subsample': np.float64(0.6957922037446137)}

In [57]:
y_train.describe()

count    3721.000000
mean        0.332438
std         0.471150
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: like_label, dtype: float64

In [58]:
##Train 
xgb_clf_score = xgb.XGBClassifier(
    enable_categorical=True,
    verbosity=0
    )
    
xgb_clf_score.set_params(**best_parameters)
xgb_clf_score.fit(X_train, y_train)

,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,np.float64(0.6196676211639822)
,device,None
,early_stopping_rounds,None
,enable_categorical,True
,eval_metric,None


In [61]:
#### Save Data and Models

time_path = "base_0"
if os.path.exists('./data_and_models/'+time_path) and os.path.isdir('./data_and_models/'+time_path):
    shutil.rmtree('./data_and_models/'+time_path)

os.makedirs('./data_and_models/'+time_path)

os.system('cp ./data/x_train.pkl '+'./data_and_models/'+time_path)
os.system('cp ./data/x_test.pkl '+'./data_and_models/'+time_path)

os.system('cp ./data/y_train.pkl '+'./data_and_models/'+time_path)
os.system('cp ./data/y_test.pkl '+'./data_and_models/'+time_path)

os.system('cp ./data/cat_ordinal_encoder.pkl '+'./data_and_models/'+time_path)
os.system('cp ./data/training_categories_array.pkl '+'./data_and_models/'+time_path)


if True:
    output_model = "./data_and_models/"+time_path+"/001_xgboost"


    if os.path.exists(output_model) and os.path.isdir(output_model):
        shutil.rmtree(output_model)
    mlflow.sklearn.save_model(xgb_clf_score, output_model+"/1")
    print(mlflow.__version__)

3.3.2


In [62]:
print(datetime.now())
print(datetime.now()-start_time)

2025-12-04 15:35:51.477200
0:02:05.813100
